**Sprint 1 -  Data collection and preprocessing**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
import pickle

In [ ]:
#reading the data set

df = pd.read_csv("/content/drive/MyDrive/datasets/autos.csv", header=0, sep=',', encoding='Latin1',)

In [ ]:
#print al the different sellers

print(df.seller.value_counts()) #remove the seller type having only 3 cars

df[df.seller != 'gewerblich'] #now all the sellers are same so we can get rid of this column

df=df.drop('seller',1)

#print al the different sellers 
print(df.offerType.value_counts())

df[df.offerType != 'Gesuch']

#remove the Offer Type having only 12 listings df [df.offerType - 'Gesuch'] #now all the offers are same so we can get rid of this column 
df=df.drop('offerType',1)

privat        371525
gewerblich         3
Name: seller, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  import sys


Angebot    371516
Gesuch         12
Name: offerType, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  from ipykernel import kernelapp as app


In [ ]:
#Cars having power less than 50ps and above 900ps seems a little suspicious,

#let's remove them and see what we've got now

print(df.shape)

df=df[(df.powerPS > 50) & (df.powerPS < 900)]

print(df.shape) #around 50000 cars ahave been removed which could have inrouduced error to our data

#simlarly, filtering our the cars having registeration years not in the mentioned range #print(df.shape)

df = df[(df.yearOfRegistration >= 1950) & (df.yearOfRegistration < 2017)] 
print(df.shape)

#not much of a difference but still, 10000 rows have been reduced. it's better to #get rid of faulty data instead of keeping them just to increase the size.

(371528, 18)
(319709, 18)
(309171, 18)


In [ ]:
#removing irrelevant columns which are either the same for all the cars in teh dataset, or can #introduce bias, so removing them too.

df.drop(['name', 'abtest', 'dateCrawled', 'nrOfPictures', 'lastSeen','postalCode', 'dateCreated'], axis='columns', inplace=True)

In [ ]:
#dropping the duplicates from the dataframe and stroing it in a new df. #here all rows having same value in all the mentioned columns will be deleted and by default,
#only first occurance of any such row is kept
new_df = df.copy()
new_df = new_df.drop_duplicates([ 'price', 'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model', 'kilometer', 'monthOfRegistration', 'fuelType' ,'notRepairedDamage'])

In [ ]:
#As the dataset contained some german words for many features, cahnging them to english
new_df.gearbox.replace(('manuell', 'automatik'), ('manual', 'automatic'), inplace=True)
new_df.fuelType.replace(('benzin', 'andere', 'elektro'), ('petrol', 'others', 'electric'), inplace=True)
new_df.vehicleType.replace(('kleinwagen', 'cabrio', 'kombi', 'andere'), ('small car', 'convertible', 'combination', 'others'), inplace=True)
new_df.notRepairedDamage.replace(('ja', 'nein'), ('Yes', 'No'), inplace=True)

In [ ]:
#### Removing the outliers
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]
#Filling NaN values for columns whose data might not be there with the information provider,
# which might lead to some variance but our model
#but we will still be able to give some estimate to the user
new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [ ]:
new_df.to_csv("autos_preprocesssed.csv")

In [ ]:
#Columns which contain categorical values, which we'll need to convert via label encoding
labels = ['gearbox', 'notRepairedDamage', 'model', 'brand', 'fuelType', 'vehicleType']
#looping over the labels to do the label encoding for all at once and
#saving the LABEL ENCODING FILES
mapper = {}
for i in labels:
  mapper[i] = LabelEncoder()
  mapper[i].fit(new_df[i])
  tr = mapper[i].transform(new_df[i])
  np.save(str('classes'+i+' .npy'), mapper[i].classes_)
  print(i, ":",mapper[i])
  new_df.loc[:, i + '_labels'] = pd.Series (tr, index=new_df.index)

#Final data to be put in a new dataframe called "LABELED",
labeled = new_df[['price','yearOfRegistration','powerPS','kilometer','monthOfRegistration']+[x+"_labels" for x in labels]]
print(labeled.columns)

gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


In [ ]:
#Storing price in Y and rest of the datain X
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values
#need to reshape the y values
Y = Y.reshape(-1,1)

**Sprint 2 - Model building**

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 3)

In [ ]:
#Model building and Fitting
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
regressor = RandomForestRegressor (n_estimators=1000, max_depth=15, random_state=34)
#fitting the model
regressor.fit(X_train, np.ravel (Y_train, order='C'))
#predicting the values fo test test
y_pred = regressor.predict(X_test)
#printing the Accuraccy for test set
print(r2_score (Y_test,y_pred))

0.860497492549767


In [ ]:
regressor.predict([[  1995,    102, 150000,     10,      1,      1,    11,     2,
            7,      4]])

array([694.33116734])

In [ ]:
#saving model for future use.
filename ='resale_model.sav'

pickle.dump(regressor, open(filename, 'wb'))